<a href="https://colab.research.google.com/github/JoshJingtianWang/TCGA-Splicing-Data-Cleaning-and-Filtering/blob/main/mutationtocandidates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Converting case IDs

In [ ]:
#############################################
#ABANDON BELOW. NO LONGER USING TCGA MUT DATA
#############################################

srun -c 4 --pty --x11 /bin/bash -i #enable x forwarding
cd /dfs5/bio/jingtiw2/SpitaleLab/SplicingTCGA/mutationworkingfiles


#get list of case IDs
import pandas as pd
ssmdf=pd.read_csv('ssm_case_id.txt', sep='\t',header=None) #read the table containing ssm ID (col0) and corresponding case IDs (col1)
caseIDs=list(set(ssmdf.iloc[:,1].tolist())) #list of UNIQUE case IDs

with open('caseIDs.txt', 'w+') as f: # write caseIDs to file for use in R
     for items in caseIDs:
         f.write('%s\n' %items)
     print("File written successfully")

In [ ]:
#############################################
#ABANDON BELOW. NO LONGER USING TCGA MUT DATA
#############################################

#convert the case UUIDs from 2019 to TCGA barcodes using the TCGAUtils package in R
#(e.g. TCGA-CH-5767-01A-11R-1789-07.07f49527-0833-481f-b172-cb67601f8b03.aligned to just TCGA-CH-5767)
module load R
R
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("TCGAutils")
library(TCGAutils)
library(curatedTCGAData)
library(MultiAssayExperiment)
library(RTCGAToolbox)
library(BiocFileCache)
library(rtracklayer)
library(R.utils)

getwd()
dir="/dfs5/bio/jingtiw2/SpitaleLab/SplicingTCGA/workingfiles"
setwd(dir)
list.files(dir)
caseIDs <- read.table("caseIDs.txt",header=FALSE) #read list of case IDs generated from python
caseIDslist <- caseIDs[,1] #turn table into list
casebarcodes <- UUIDtoBarcode(caseIDslist, from_type = "case_id") #convert case IDs into TCGA barcodes
write.table(casebarcodes, file = 'casebarcodes.txt',sep = "\t",quote = FALSE,row.names = F) #write barcodes to file for use in python

In [ ]:
#convert numbered labels to TCGA barcodes to UUID

##################################################################
#NO LONGER NEEDED
#make the UUID-to-barcode dictionary
IDbarcode=pd.read_csv('casebarcodes.txt',sep='\t') #read table generated in R
UUIDtobarcodedict=dict(IDbarcode.values)
##################################################################

##################################################################
#NO LONGER NEEDED
#get single line header file containing only the TCGA barcodes
with open('headers') as f:
    barcodes = f.readline().strip() #this reads only first line https://stackoverflow.com/questions/1904394/read-only-the-first-line-of-a-file  .strip() removes \t and \n

barcodes=barcodes.split('\t') #turn line into list
barcodes=barcodes[6:] #remove gene info at the start

#barcodes=[s.strip() for s in barcodes]

#make the barcode-to-label dictionary
i=1
barcodetolabeldict={}
for barcode in barcodes:
  label='patient'+str(i)
  barcodetolabeldict[barcode]=label
  i+=1
##################################################################

##################################################################
#make the label-to-barcode dictionary (keeping only cancer samples)
with open('headers') as f:
    next(f) #reads second line https://www.reddit.com/r/Python/comments/8i02uo/reading_only_the_second_line_of_a_text_file/
    barcodes2 = f.readline().strip().split('\t')
barcodes2=barcodes2[6:]

import re
i=1
labeltobarcodedict={}
for barcode in barcodes2:
  label='patient'+str(i)
  type=re.sub(r'^TCGA-\w\w-\w\w\w\w-|[A-Z]-\d\d[A-Z]-\w{4}-\d{2}\..+$', '', barcode)
  type=int(type.lstrip('0'))
  if type < 10:
    trimmedbarcode=re.sub(r'-\d\d[A-Z]-\d\d[A-Z]-\w{4}-\d{2}\..+$', '', barcode)
    labeltobarcodedict[label]=trimmedbarcode
  i+=1


len(labeltobarcodedict.keys())
len(labeltobarcodedict.values())
#Out: 9310
##################################################################


#Process mutation table from UCSC Xena (GDC-PANCAN.mutect2)

In [ ]:
#compare cases from Xena vs cases from Ratsch

#get Xena cases
import pandas as pd
xenadf=pd.read_csv('GDC-PANCAN.mutect2_snv.tsv', sep='\t')
xenacases=xenadf['Sample_ID'].tolist()
xenacases=list(set(xenacases))

import re
xenacasestrimmed=[]
for id in xenacases:
  trimmedid=re.match(r'^TCGA-\w\w-\w\w\w\w', id)
  xenacasestrimmed.append(trimmedid.group(0))

len(xenacasestrimmed)
#Out:10182

#get Ratsch cases
with open('headers') as f:
    ratschcases = f.readline().strip() #this reads only first line https://stackoverflow.com/questions/1904394/read-only-the-first-line-of-a-file

ratschcases=ratschcases.split('\t') #turn line into list
ratschcases=ratschcases[6:] #remove gene info at the start
len(ratschcases)
#Out: 10019

#get manually queried TCGA cases
IDbarcode=pd.read_csv('casebarcodes.txt',sep='\t')
TCGAcases=IDbarcode['submitter_id'].tolist()
TCGAcases=list(set(TCGAcases))


#compare shared cases
len(set(xenacasestrimmed).intersection(ratschcases))
#Out: 8878
len(set(TCGAcases).intersection(ratschcases))
#Out: 8633

Verdict: GDC-PANCAN.mutect2 from UCSC Xena has more data. Therefore use over manually queried TCGA mut data.

In [ ]:
#PROBABLY USELESS

#query the Xena API for mutations per sample
#https://gist.github.com/acthp/f9828f57eb795404eddb85006375af8d
#https://github.com/ucscXena/xenaPython

import xenaPython as xena

host = xena.PUBLIC_HUBS['gdcHub']
cohort = 'GDC Pan-Cancer (PANCAN)'
samples = xena.cohort_samples(host, cohort, None)
samples[0: 10]
dataset = 'GDC-PANCAN.mutect2_snv.tsv'
mutation_columns = xena.sparse_data(host, dataset, samples, xenagenes)


#Making Mut-Count tables for R

make dict for mutation ranked by #of eligible events it causes in a patient

In [ ]:
##################################################################
##################################################################

#make dict for mutation ranked by #of eligible events it causes in a patient
#ssm_case_id.txt


#first, reproduce the patient:#of eligible events dictionary
#load dictionary from file
import pickle
with open('candidatepvalues.pkl', 'rb') as f:
  mydict=pickle.load(f)

#getting test count for multiple testing correction
testcount=0
for event in mydict:
  testcount=testcount+len(mydict[event])-6

#multiple testing correction
import re
for event in mydict:  
  for patient in list(mydict[event]): #https://stackoverflow.com/questions/11941817/how-to-avoid-runtimeerror-dictionary-changed-size-during-iteration-error
    pvalue=mydict[event][patient]
    if re.match(r'^patient\d+', patient):
    #if patient.startswith('patient'):
      pvalueadj=pvalue*testcount
      if pvalueadj >= 0.001:
        mydict[event].pop(patient)

#getting list of patients (including duplicates)  who passed the filters
patientlist=[]
for event in mydict:
  for patient in mydict[event]:
    if re.match(r'^patient\d+', patient):
    #if patient.startswith('patient'):
      patientlist.append(patient)
print(len(patientlist))

#dictionary of {patient:count of events that passed filters}
patientdict={}
for i in patientlist:
  patientdict[i] = patientdict.get(i, 0) + 1 #https://stackoverflow.com/questions/3496518/using-a-dictionary-to-count-the-items-in-a-list/6582852#6582852




#############################################
#ABANDON BELOW. NO LONGER USING TCGA MUT DATA
#############################################

#get unique list of ssm IDs
cd /dfs5/bio/jingtiw2/SpitaleLab/PTM/PTM_list/TCGA #change directory
IDdf=pd.read_csv('ssm_case_id.txt', sep='\t',header=None) #read the table containing ssm ID (col0) and corresponding case IDs (col1)
ssmIDs=list(set(IDdf.iloc[:,0].tolist())) #list of UNIQUE ssm IDs

#turn the unique ssm ID into dictionary with key:empty list
eventmutdict1={}
for ssmID in ssmIDs:
  eventmutdict1[ssmID]=[]

#appending event count of each patient to the dictionary
with open("ssm_case_id.txt") as fp:
  for line in fp:
    line=line.split('\t') #turn line into list
    line=[s.strip() for s in line] ##remove potential newline character https://stackoverflow.com/questions/12533955/string-split-on-new-line-tab-and-some-number-of-spaces
    ssmID=line[0]
    caseUUID=line[1]
    casebarcode=UUIDtobarcodedict[caseUUID]
    try:        #try/except in case case ID not found in petientdict?
      caselabel=barcodetolabeldict[casebarcode]
    except:
      continue
    eventmutdict1[ssmID].append(patientdict[caselabel])

#make a copy to be used later for the other graph
import copy
eventmutdict2 = copy.deepcopy(eventmutdict1) #https://stackoverflow.com/a/2465951

#remove from eventmutdict1 where value len == 0
for event in list(eventmutdict1):
  if len(eventmutdict1[event])==0:
    eventmutdict1.pop(event)

#get average # of events a mutation causes in a patient
for ssmID in eventmutdict1:
  eventcountlist=eventmutdict1[ssmID]
  eventmutdict1[ssmID]=sum(eventcountlist) / len(eventcountlist)

#turn dictionary into a dataframe for further processing in R https://stackoverflow.com/questions/18837262/convert-python-dict-into-a-dataframe
eventcount1df=pd.DataFrame(eventmutdict1.items(), columns=['Mutation', 'AverageEventCount'])
with open('eventcountdf_for_R.csv', 'w') as file:
     eventcount1df.to_csv(file, sep='\t')


stacked barplot of patient event count + mutation count


In [ ]:
##################################################################
##################################################################
#stacked barplot of patient event count + mutation count

#list of unique patients who passed the filters
patientlistunique=list(set(patientlist))
print(len(patientlistunique))
#Out 10019

#get total unfiltered patient count
with open('headers') as f:
    ratschcases = f.readline().strip() #this reads only first line https://stackoverflow.com/questions/1904394/read-only-the-first-line-of-a-file
ratschcases=ratschcases.split('\t') #turn line into list
ratschcases=ratschcases[6:] #remove gene info at the start
len(ratschcases)
#Out:10019

#dictionary of {patient:count of events that passed filters}
patientdict={} 
for i in patientlist:
  patientdict[i] = patientdict.get(i, 0) + 1 #https://stackoverflow.com/questions/3496518/using-a-dictionary-to-count-the-items-in-a-list/6582852#6582852

#because some "patients" are actually replicates of one patient, need to condense those replicates by averaging the event counts
patientdict2={}
for barcode in labeltobarcodedict.values(): #build dict of lists
  patientdict2[barcode]=[]
for label in patientdict: #append event counts to each key
  try: #try/except block because some labels in patientdict are not found in patientdict2
    barcode=labeltobarcodedict[label]
  except:
    continue
  patientdict2[barcode].append(patientdict[label])
for barcode in patientdict2: #taking average
  eventcounts=patientdict2[barcode]
  patientdict2[barcode]=sum(eventcounts) / len(eventcounts)


####################################
#get patient mutation count from queried TCGA data (ABANDONED)
patientmutcountdict={}
with open("ssm_case_id.txt") as fp:
  for line in fp:
    line=line.split('\t') #turn line into list
    line=[s.strip() for s in line] ##remove potential newline character https://stackoverflow.com/questions/12533955/string-split-on-new-line-tab-and-some-number-of-spaces
    ssmID=line[0]
    caseUUID=line[1]
    casebarcode=UUIDtobarcodedict[caseUUID]
    try:        #try/except in case case ID not found in petientdict?
      caselabel=barcodetolabeldict[casebarcode]
    except:
      continue
    patientmutcountdict[caselabel] = patientmutcountdict.get(caselabel, 0) + 1
####################################

####################################
#get patient mutation count using UCSC Xena GDC-PANCAN.mutect2 data, using this instead
xenadf=pd.read_csv('GDC-PANCAN.mutect2_snv.tsv', sep='\t')
xenacases=xenadf['Sample_ID'].tolist()
patientmutcountdict2={}
for i in xenacases:
  match=re.match(r'^TCGA-\w\w-\w\w\w\w-\w\w\w', i)
  trimmedid=match.group(0)
  patientmutcountdict2[trimmedid] = patientmutcountdict2.get(trimmedid, 0) + 1 #getting patient mutation count from Xena data

#turning patient mutation count into a pandas df
xenamutcountdf=pd.DataFrame(patientmutcountdict2.items(), columns=['Patient', 'MutCount'])
####################################


#convert dict into pandas dataframe
#patientmutdf=pd.DataFrame(patientmutcountdict.items(), columns=['Patient', 'MutCount']) #https://stackoverflow.com/questions/18837262/convert-python-dict-into-a-dataframe
patienteventdf=pd.DataFrame(patientdict2.items(), columns=['Patient', 'EventCount'])

#use pandas merge to join the two dataframes https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
patienteventmutdf = pd.merge(patienteventdf, xenamutcountdf, how="outer", on=['Patient'])

patienteventmutdf=patienteventmutdf.dropna()

#writing table to file
with open('eventmutcount_for_R.csv', 'w') as file:
     patienteventmutdf.to_csv(file, sep='\t')



####################################

#stacked barplot of patient event count + spliceosome mutation count
#get list of spliceosome gene names...



#stacked barplot of patient event count + spliceosome mutation count


# Generate plot in R

In [ ]:
#scatter plot of mut count vs event count
srun -c 4 --pty --x11 /bin/bash -i
#R
module load R
R

library(ggplot2)
eventmutcount <- read.table("eventmutcount_for_R.csv",header=TRUE,sep='\t')
head(eventmutcount)
# Basic scatter plot
ggplot(eventmutcount, aes(x=MutCount, y=EventCount)) + geom_point()

#Making Spliceosome-only mutation count table for R

In [ ]:
#spliceosome
#spliceosome protein list downloaded from http://spliceosomedb.ucsc.edu/
import pandas as pd
spliceosomedf=pd.read_csv('spliceosomeprotein.csv', sep=',')
spliceosomelist=spliceosomedf['Gene Symbol'].tolist()

#subset xena mutation data with spliceosome protein list
#https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
xenadf=pd.read_csv('GDC-PANCAN.mutect2_snv.tsv', sep='\t')
xenasplicedf=xenadf.loc[xenadf['gene'].isin(spliceosomelist)]
xenasplicecases=xenasplicedf['Sample_ID'].tolist()

#get spliceosome mutation data
import re
patientmutcountdict3={}
for i in xenasplicecases:
  match=re.match(r'^TCGA-\w\w-\w\w\w\w-\w\w\w', i)
  trimmedid=match.group(0)
  patientmutcountdict3[trimmedid] = patientmutcountdict3.get(trimmedid, 0) + 1 #getting patient mutation count from Xena data
xenasplicemutcountdf=pd.DataFrame(patientmutcountdict3.items(), columns=['Patient', 'MutCount'])


#use pandas merge to join the two dataframes https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
patienteventsplicemutdf = pd.merge(patienteventdf, xenasplicemutcountdf, how="outer", on=['Patient'])
patienteventsplicemutdf=patienteventsplicemutdf.dropna()

#writing table to file
with open('eventsplicemutcount_for_R.csv', 'w') as file:
     patienteventsplicemutdf.to_csv(file, sep='\t')

In [ ]:
#U2, U5 complexes
U2df=pd.read_csv('U2splice.csv', sep=',')
U2list=U2df['Gene Symbol'].tolist()
U5df=pd.read_csv('U5splice.csv', sep=',')
U5list=U5df['Gene Symbol'].tolist()
U2U5list=U2list+U5list


#subset xena mutation data with spliceosome protein list
#https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
xenadf=pd.read_csv('GDC-PANCAN.mutect2_snv.tsv', sep='\t')
xenaU2U5df=xenadf.loc[xenadf['gene'].isin(U2U5list)]
xenaU2U5cases=xenaU2U5df['Sample_ID'].tolist()

import re
patientmutcountdict3={}
for i in xenaU2U5cases:
  match=re.match(r'^TCGA-\w\w-\w\w\w\w-\w\w\w', i)
  trimmedid=match.group(0)
  patientmutcountdict3[trimmedid] = patientmutcountdict3.get(trimmedid, 0) + 1 #getting patient mutation count from Xena data
xenaU2U5mutcountdf=pd.DataFrame(patientmutcountdict3.items(), columns=['Patient', 'MutCount'])


# Generate plot in R

In [ ]:
#scatter plot of mut count vs event count
srun -c 4 --pty --x11 /bin/bash -i
#R
module load R
R

library(ggplot2)
eventsplicemutcount <- read.table("eventsplicemutcount_for_R.csv",header=TRUE,sep='\t')
head(eventsplicemutcount)

png('spliceosomemutvevent.png', width = 1200, height = 1000, res = 200)
# Basic scatter plot
ggplot(eventsplicemutcount, aes(x=MutCount, y=EventCount)) + geom_point()
dev.off()

#Verifying event counts




In [ ]:
srun -c 4 --pty --x11 /bin/bash -i

import pandas as pd

patienteventdf.sort_values(by='EventCount', ascending=False)
eventcountsorted = patienteventdf.sort_values(by='EventCount')

               Patient  EventCount
8444  TCGA-20-1683-01A       256.0
376   TCGA-55-7576-01A       245.0
4630  TCGA-29-1697-01A       244.0
5065  TCGA-24-2261-01A       237.0
7039  TCGA-24-1603-01A       231.0
8697  TCGA-BP-4974-01A       229.0
...                ...         ...
3970  TCGA-BK-A0CC-01B         8.0
7485  TCGA-BK-A139-01C         8.0
3968  TCGA-44-2665-01B         7.0
2719  TCGA-OR-A5L9-01A         5.0
3622  TCGA-06-0168-01A         4.0

patientlist=eventcountsorted['Patient'].tolist()
mylist=patientlist[:5]+patientlist[-1:]

mylist=['event_id',
  'TCGA-20-1683',
  'TCGA-06-0168']


verifydf = pd.read_csv('/dfs5/bio/jingtiw2/SpitaleLab/SplicingTCGA/candidateworkingfiles/threeprimePSItrimmedheader', usecols = mylist, low_memory = True, sep='\t')
verifydf=verifydf.drop(verifydf.index[0]) #drop first row

verifydf=verifydf.set_index('event_id')

with open('verifydf.csv', 'w') as file:
    #changing NaN to NULL since to_csv writes NaN as empty
     verifydf.to_csv(file, sep='\t', na_rep='NULL') #https://stackoverflow.com/questions/50890989/pandas-changing-the-format-of-nan-values-when-saving-to-csv

#then, run verifydf.csv through the getcandidates.py pipeline
#Out: {'patient2': 256, 'patient1': 4}
#Results confirmed to be correct\

#rank mutated (Xena) genes by count of events they cause

In [ ]:
#get table of mut-patient (Xena)
#mutdict={mut:[patient1, patient2...], mut2:[]}
import pandas as pd
xenadf=pd.read_csv('GDC-PANCAN.mutect2_snv.tsv', sep='\t')
#xenadf['mutlabel'] = xenadf.index #turn index into column
#xenadf['mutlabel'] = 'mut' + df['mutlabel'].astype(str) #prepend 'mut' to column
xenadf=xenadf[['Sample_ID','gene']].drop_duplicates(subset=['Sample_ID', 'gene'], keep='last') #subset two columns, remove duplicates
xenadf.rename({'Sample_ID': 'Patient'}, axis=1,inplace=True) #rename column

#get table of patient-event count
#eventcount={patient:count,patient2:count2}
patienteventdf


####################################
#join two tables
#use pandas merge to join the two dataframes https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
merged = pd.merge(patienteventdf, xenadf, how="inner", on=['Patient'])
#group dataframe by mutation and get average of event counts
#https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.mean.html
generank=merged.groupby('gene')['EventCount'].mean()

with open('mutgenesvsevemt_for_R.csv', 'w') as file:
     generank.to_csv(file, sep='\t')

####################################
#label spliceosome genes in the table
#https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
generankdf=pd.read_csv('mutgenesvsevemt_for_R.csv', sep='\t')

generankdf_sorted = generankdf.sort_values(by=['EventCount'], ascending=False) #sort


#list of gene names form HGNC
genenames=pd.read_csv('genenames.txt', sep='\t')
geneseries=genenames[genenames["Locus type"]=="gene with protein product"].loc[:,'Approved symbol'] #get only protein-coding genes, then get only gene symbols
genelistdf=pd.DataFrame({'index':geneseries.index, 'gene':geneseries.values})
merged = pd.merge(generankdf_sorted, genelistdf, how="inner", on=['gene'])

generankB100=merged.iloc[-100:].loc[:,'gene'] #select bottom 50 genes
generankT100=merged.iloc[:100].loc[:,'gene'] #select top 50 genes
generankbg=merged.sample(200).loc[:,'gene'].tolist() #random selection

#after inputting genelists into http://geneontology.org/, no sig results were found


spliceosomedf=pd.read_csv('spliceosomeprotein.csv', sep=',')
spliceosomelist=spliceosomedf['Gene Symbol'].tolist()

U2df=pd.read_csv('U2splice.csv', sep=',')
U2list=U2df['Gene Symbol'].tolist()
U5df=pd.read_csv('U5splice.csv', sep=',')
U5list=U5df['Gene Symbol'].tolist()
U2U5list=U2list+U5list

generanksplice=generankdf.loc[generankdf['gene'].isin(spliceosomelist)] #isin list
generanknotsplice=generankdf.loc[~generankdf['gene'].isin(spliceosomelist)] #is not in list
generanksplice["EventCount"].mean()
generanknotsplice["EventCount"].mean()

generanksU2U5=generankdf.loc[generankdf['gene'].isin(U2U5list)] #isin list
generanknotU2U5=generankdf.loc[~generankdf['gene'].isin(U2U5list)] #is not in list
U2U5array=generanksU2U5['EventCount'].values
notU2U5array=generanknotU2U5['EventCount'].values

from scipy.stats import ttest_ind
t, p = ttest_ind(U2U5array, notU2U5array, equal_var=False)
#In [12]: t
#Out[12]: 1.1652359773740997
#In [13]: p
#Out[13]: 0.2537011305849394


Generate plot in R

In [ ]:
#scatter plot of mut count vs event count
srun -c 4 --pty --x11 /bin/bash -i
#R
module load R
R

mutgenevsevent <- read.table("mutgenesvsevemt_for_R.csv",header=TRUE,sep='\t')
head(mutgenevsevent)

eventcountT500<-head(mutgenevsevent[order(eventcount$Count,decreasing=TRUE),],500) #sorting by Count, top 500 events


# Basic histogram http://www.sthda.com/english/wiki/ggplot2-histogram-plot-quick-start-guide-r-software-and-data-visualization
ggplot(eventcountnozero, aes(x=Count)) + geom_histogram()
# Change the width of bins
ggplot(eventcountnozero, aes(x=Count)) + 
  geom_histogram(binwidth=1)
# Change colors
p<-ggplot(eventcountnozero, aes(x=Count)) + 
  geom_histogram(color="black", fill="white")
p


#rank mutation IDs (TCGA) by count of events they cause (TCGA mutations that I have may only contain RBP mutations)

In [ ]:
####################################
#get patient mutation count from queried TCGA data
#generate table that is: ssmid  casebarcode
ssm_case_iddf=pd.read_csv('ssm_case_id.txt', sep='\t', header=None)
ssm_case_iddf.columns = ["ssm_id", "case_uuid"]
UUIDtobarcodedf=pd.DataFrame(UUIDtobarcodedict.items(), columns=['case_uuid', 'Barcode']) 
ssm_case_merged=pd.merge(ssm_case_iddf, UUIDtobarcodedf, how="inner", on=['case_uuid']) #converting case UUID to barcode by merging two df
ssm_case_merged = ssm_case_merged[['Barcode','ssm_id']]
ssm_case_merged.columns = ['Case','ssm_id']


####################################
patienteventdf=pd.DataFrame(patientdict2.items(), columns=['Case', 'EventCount'])

####################################

#join mut table vs event table
#use pandas merge to join the two dataframes https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
merged = pd.merge(patienteventdf, ssm_case_merged, how="inner", on=['Case'])
#group dataframe by mutation and get average of event counts
#https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.mean.html
generank=merged.groupby('ssm_id')['EventCount'].mean()

with open('TCGAmutvsevent_for_R.csv', 'w') as file:
     generank.to_csv(file, sep='\t')

####################################
#compare spliceosome vs non-spliceosome
#label spliceosome genes in the table
#https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
generankdf=pd.read_csv('TCGAmutvsevent_for_R.csv', sep='\t')



ssmdf=pd.read_csv('ssm_case_id.txt', sep='\t',header=None)
mutfreqdf=pd.read_csv('mut_freq.table', sep='\t',header=None)
mutfreqdf = mutfreqdf[[0,2]].drop_duplicates()
mutfreqdf.columns = ['ssm_id','gene']

merged = pd.merge(mutfreqdf, generankdf, how="inner", on=['ssm_id'])
generankdf=merged.groupby('gene')['EventCount'].mean()
generankdf=pd.DataFrame({'gene':generankdf.index, 'meaneventcount':generankdf.values}) #convert series to df
generankdf_sorted = generankdf.sort_values(by=['meaneventcount'], ascending=False) #sort
generankB50=generankdf_sorted.iloc[-50:].loc[:,'gene'] #select bottom 50 genes
generankT50=generankdf_sorted.iloc[:50].loc[:,'gene'] #select top 50 genes
generankdf_sorted.sample(50) #random selection





spliceosomedf=pd.read_csv('spliceosomeprotein.csv', sep=',')
spliceosomelist=spliceosomedf['Gene Symbol'].tolist()

U2df=pd.read_csv('U2splice.csv', sep=',')
U2list=U2df['Gene Symbol'].tolist()
U5df=pd.read_csv('U5splice.csv', sep=',')
U5list=U5df['Gene Symbol'].tolist()
U2U5list=U2list+U5list


generanksplice=merged.loc[merged['gene'].isin(spliceosomelist)] #isin list
generanknotsplice=merged.loc[~merged['gene'].isin(spliceosomelist)] #is not in list
splicearray=generanksplice["EventCount"].values
notsplicearray=generanknotsplice["EventCount"].values

from scipy.stats import ttest_ind
t, p = ttest_ind(splicearray, notsplicearray, equal_var=False)

generanksU2U5=merged.loc[merged['gene'].isin(U2U5list)] #isin list
generanknotU2U5=merged.loc[~merged['gene'].isin(U2U5list)] #is not in list
U2U5array=generanksU2U5['EventCount'].values
notU2U5array=generanknotU2U5['EventCount'].values





In [ ]:
#scatter plot of mut count vs event count
srun -c 4 --pty --x11 /bin/bash -i
#R
module load R
R

mutgenevsevent <- read.table("mutgenesvsevemt_for_R.csv",header=TRUE,sep='\t')
head(mutgenevsevent)

eventcountT500<-head(mutgenevsevent[order(eventcount$Count,decreasing=TRUE),],500) #sorting by Count, top 500 events


# Basic histogram http://www.sthda.com/english/wiki/ggplot2-histogram-plot-quick-start-guide-r-software-and-data-visualization
ggplot(eventcountnozero, aes(x=Count)) + geom_histogram()
# Change the width of bins
ggplot(eventcountnozero, aes(x=Count)) + 
  geom_histogram(binwidth=1)
# Change colors
p<-ggplot(eventcountnozero, aes(x=Count)) + 
  geom_histogram(color="black", fill="white")
p

#Investigate mutations of patients with the RPM10 event